In [2]:
import re

from PyPDF2 import PdfReader
import pandas as pd
fern_species = pd.read_excel("Species list.xlsx")


In [47]:
from tqdm.auto import tqdm

conditional_newline = lambda sp: '\s*\n?'.join(sp.split(' ')).replace('.', '\.').replace('(', '\(').replace(')', '\)').replace('[', '\[').replace(']', '\]')
sp_descriptions = pd.DataFrame()
for fam in tqdm(fern_species.Family.unique()):
    text = ""
    with open(f"descriptions/{fam}.pdf", "rb") as pdf_file:
        read_pdf = PdfReader(pdf_file)
        for i, page in enumerate(read_pdf.pages):
            if i < 5:
                continue
            tx = page.extract_text()
            if tx.split("\n")[0].startswith('References'):
                # print(f"Breaking at page {i}")
                break
            text += page.extract_text()
    text = re.sub(r'[^\S\r\n]+', ' ', text)

    fam_species = fern_species[fern_species.Family.eq(fam)].Species.unique()
    sp_dict = {}
    for i, species in enumerate(fam_species):
        species = species
        if (cnt := len(re.findall(conditional_newline(species), text))) == 0:
            print(f"{fam} | {species} wrongly appearing (count: {cnt})")
            foo
        # start = text.find(species)
        # start is the first character of the species name assuming to find it in the text
        # with potentially some '\n' characters in between
        start = re.search(conditional_newline(species), text).start()
        end = [re.search(conditional_newline(sp), text).start() for sp in fam_species]
        end = min([e for e in end if e and e > start] + [len(text)])
        name_sp = species.split(')')[0] + ')'
        if name_sp not in sp_dict.keys():
            sp_dict[name_sp] = text[start:end]
        else:
            sp_dict[name_sp] += '\n' + text[start:end]

    sp_descriptions = pd.concat([sp_descriptions, pd.DataFrame(sp_dict.items(), columns=['Species', 'Description']).assign(Family=fam)], ignore_index=True)


def desc_to_sections(desc):
    """
    Split the description into sections:
    "Etymology", "Vernacular name", "Distribution", "Altitudinal range", "Biostatus", "Habitat", "First record ", "Recognition ", "Cytology", "Hybridisation", "Notes "
    not all the sections will be present in all descriptions, if present, they start with the section name followed by a colon, up to the next named section or the end of the description
    """
    sections = ["Etymology", "Vernacular name", "Distribution", "Altitudinal range", "Biostatus", "Habitat", "First record", "Recognition", "Cytology", "Hybridisation", "Notes"]
    section_dict = {}
    for i, section in enumerate(sections):
        if i == len(sections) - 1:
            section_dict[section] = desc
        else:
            start = re.search(fr"{section}s?:", desc)
            start = start.end() if start else -1
            if start == -1:
                continue
            end = re.search(fr"{sections[i+1]}s?:", desc)
            end = end.start() if end else -1

            this_desc = desc[start:end].strip()

            section_dict[section] = re.sub(r'\n+', ' ', this_desc.strip())
    return pd.Series(section_dict)

extra = sp_descriptions.Description.apply(desc_to_sections)  
sp_descriptions = pd.concat([sp_descriptions, extra], axis=1).drop(columns='Description')

  0%|          | 0/32 [00:00<?, ?it/s]

In [50]:
sp_descriptions.to_excel("fern_descriptions.xlsx", index=False)

# Extract features

In [27]:
sp_descriptions = pd.read_excel("fern_descriptions.xlsx").set_index('Species')

In [8]:
extra_features_df = pd.read_excel("../Words before and after traits_v2.xlsx", sheet_name="FernPrecedingWords")
extra_features_df = extra_features_df[
    :extra_features_df[extra_features_df.Rhizome.str.startswith('Sentences that talk a', na=False)].index[0]
    ].map(lambda s:s.lower().strip() if type(s) == str else s)

extra_features = extra_features_df.to_dict('list')
extra_features = {k.title().replace(' ', ''): [x for x in v if str(x) != 'nan'] for k, v in extra_features.items()}

In [18]:
unit = '[m|c|d|µ]?m'
number = r"(\d+\.?\d*)"
full_regex = rf"(({number}\s?-\s?)?{number})?({number}\s?-\s?)?{number}\s*{unit}(-wide)?(-long)?"
full_regex = full_regex + r'(?=[\s\.,;])'

In [19]:
print(full_regex)

(((\d+\.?\d*)\s?-\s?)?(\d+\.?\d*))?((\d+\.?\d*)\s?-\s?)?(\d+\.?\d*)\s*[m|c|d|µ]?m(-wide)?(-long)?(?=[\s\.,;])


In [5]:
tmp = []
anomalies = set() # species with anomalies

def extract_features(i, feats:list):
	features = {}
	for feat in feats:
		if len(feat) < 1:
			continue
		feat = feat.replace(',', ' ')
		measures = re.finditer(full_regex, feat)
		for measure in measures:
			found = None
			for key, values in extra_features.items():

				feat = feat[:-1] if feat[-1] in ['.', ';'] else feat # remove any of .; at the end of the sentence
				matched_word = list(re.finditer( r'\b('+ '|'.join([w for w in set(values)]) + r')\b', feat.lower()))
				
				if any(matched_word):	
					if key != 'Stature': # TODO: Caso up to 3m, petiole 2mm???
						matched_word = [w for w in matched_word if w.span()[0] < measure.span()[0]]
						if not any(matched_word):
							continue # if not stature and measure appears before the word, skip
					matched_word = sorted(matched_word, key=lambda word: word.span()[1] - measure.span()[0])[0]

					this_distance = abs(matched_word.span()[1] - measure.span()[0])
					# this_distance = abs(word_match_position - measure_position)
					if found:
						if (any([w in feat.lower() for w in ['achene', 'cypsela']]) and {key, found[0]} == {'FruitSize', 'SeedSize'}) or\
						   (any([w in feat.lower() for w in ['stigma-style']]) and {key, found[0]} == {'StigmaSize', 'StyleSize'}) or\
						   (any([w in feat.lower() for w in ['floret']]) and {key, found[0]} == {'RayFloretsSize', 'DiskFloretSize'}):
							pass
							# print(f'OK>> Multiple features found ({found}, {key}) in "{feat}"')
						else:
							if this_distance >= found[1]:
								continue
							else:
								# print(found, key, matched_word, measure)
								features[found[0]].remove(found[2])
							anomalies.add(i)
					found = (key, this_distance, measure.group().strip())
					
					if key in features:
						features[key].append(measure.group().strip())
					else:
						features[key] = [measure.group().strip()]
	return pd.Series(features)



In [4]:
def string_preprocessing(s):
	s = s.replace('\xa0', ' ').replace('×', 'x').replace('–', '-').replace('·', '.') # remove non-breaking space and replace multiplication sign with x
	s = re.sub(r'(?<=xcluding)\s+[\w-]+', ' ', s) # remove each word following "excluding" (Mericarps (excluding style) 2.5-3.0 mm should point to "Mericarps")
	s = re.sub(fr'-?\(-?{number}-?\)-?', '', s) # remove all parentesis surrounding a number and the number inside (e.g. (-1.5) --> **)
	# s = s.replace('--', '-').replace('-.', '-').replace('..', '.')
	# s = s.replace('(', '').replace(')', '')
	# s = re.sub(r'\s(c|ca|o)\.', ' foo ', s) # remove all ' c.'
	s = re.sub(r'(?<=\d)\s+(?=[cmd]?m)', '', s) # remove all spaces before measures (mm, cm, dm, m, these strings only if padded by a space)
	s = re.sub('m long;?', 'm-long', s) # remove space between measure and "long" (e.g. 2 mm long --> 2 mm-long)
	s = re.sub('m wide;?', 'm-wide', s) # remove space between measure and "wide" (e.g. 2 mm wide --> 2 mm-wide)
	s = re.sub(r'\s*-\s*', '-', s) # remove spaces around hyphens
	s = re.sub(r'(?<=\d)\s*\.(?=\d)', '.', s) # remove spaces before dot if followed and preceded by a number
	s = re.sub(r'(?<=\s)\.(?=\d)', '0.', s) # add a 0 before a dot if it is preceded by a space and a "not number" and followed by a number (e.g. foo .5 --> foo 0.5)
	# s = re.sub(r'(?<=[\dm])\s*x\s*(?=\d+)', 'x', s) # remove spaces around x in formulas
	s = re.sub(rf'(;\s*)({full_regex})', r' \2', s) # point to any ';' preceding a measure (full_regex) and remove it, without removing the measure
	# now all measures are supposed to have no spaces between number and unit and spaces around them

	# s = re.sub(rf'(?<=\d{unit})(\s*long,?\s*)(?={number}{unit})', r'x', s) # remove any 'long' after a measure ("2 mm long X 3 mm wide" --> "2 mm x 3 mm wide")
	# s = re.sub(rf'(?<=\d)([m|c|d]m|m(?!m))(?!x)', r'\1 ', s) # fix situation in which a measure is not followed by a space, in the case, add that space
	# s = re.sub(r'(?<![\d\sx\.-])(\d)', r' \1', s) # fix the situation in which a measure (the whole number and measure) is not preceded by a space. In the case, add a space before the measure
	# s = re.sub(rf'(?<=\s)-(?=\d)', '', s) # remove '-' at the beginning of a measure (e.g. -1.5 --> 1.5)
	# s = re.sub(r'(?<=\d\.\d+)(\.\d?)', '', s) # fix the error in which there is a doubled dot in a number (e.g. 1.5.2), in the case, remove the second dot and the eventual numbers after it
	# s = re.sub(r'(?<=[a-ln-z])-(?=\d)', ' ', s) # remove all '-' preceded by a letter (different from m) and followed by a number (e.g. to-250mm --> to 250mm)
	# s = re.sub(r'(?<![a-z])(l|I)(?=[\s\.-]|\d)', '1', s) # replace all 'l' or "I" characters which should be '1' (e.g. l.5 --> 1.5). This should be followed by a space, a dot, a hyphen, or a number and not preceded by a letter
	return s

In [48]:
features = sp_descriptions.Etymology.fillna('') + ' ' + sp_descriptions['Vernacular name'].fillna('')
features = features.apply(string_preprocessing).str.split(r'\s\.').reset_index().apply(lambda x: extract_features(x.Species, x[0]), axis=1)
features = features.map(lambda x: '; '.join(x) if not isinstance(x, float) else x)
features.index = sp_descriptions.index
features[features.notna().sum(axis=1) > 0].to_csv('processed_features_fern.csv')

/tmp/ipykernel_833904/1901678600.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  features = features.applymap(lambda x: '; '.join(x) if not isinstance(x, float) else x)


---

In [12]:
from pathlib import Path
import pandas as pd
import re


processed_path = Path.cwd().parent / "Felci/processed_features_fern.csv"
species = pd.read_excel(Path.cwd().parent / "Felci/fern_descriptions.xlsx").set_index('Species')
species['Features'] = species.Etymology.fillna('') + ' ' + species['Vernacular name'].fillna('')


processed_features = species.Features.apply(string_preprocessing).str.split(r'\s\.').reset_index().apply(lambda x: extract_features(x.Species, x.Features), axis=1)
processed_features = processed_features.map(lambda x: '; '.join(x) if not isinstance(x, float) else x)
processed_features.index = species.index


    

In [16]:
{feat: i for i, feat in enumerate(processed_features.columns)} | {'null': '#808080'}

{'FertileFronds': 0,
 'Frond': 1,
 'Habit**': 2,
 'Laminae': 3,
 'PrimaryPinnae': 4,
 'Rachis': 5,
 'Rhizome': 6,
 'SecondaryPinnae': 7,
 'Sori': 8,
 'Sporangia': 9,
 'Spores': 10,
 'SterileFronds': 11,
 'Stipe': 12,
 'TertiaryPinnae': 13,
 'Venation*': 14,
 'null': '#808080'}

In [11]:
species.Features.apply(string_preprocessing).str.split(r'\s\.').reset_index()

,Species,Features
0,Anemia phyllitidis (L.),[From the Greek phyllitidis (like Phyllitis )....
1,Athyrium filix-femina (L.),[From the Latin filix (fern) and femina (femal...
2,Athyrium otophorum (Miq.),[From the Greek otos (ear) and phorus (bearing...
3,Deparia petersenii subsp. congrua (Brack.),[From the Latin congruus (agreeable). Rhizomes...
4,Deparia petersenii (Kunze),"[From the Greek diplasios (double), a referenc..."
...,...,...
265,Cyclosorus interruptus (Willd.),[From the Latin interruptus (interrupted). Rhi...
266,Macrothelypteris torresiana (Gaudich.),[Named in honour of Luís Vaz de Torres (b. 156...
267,Pakau pennigera (G.Forst.),[From the Latin pennigerus (with feathery leav...
268,Pseudophegopteris aurita (Hook.),"[From the Latin auritus (long-eared), a refere..."
